<a href="https://colab.research.google.com/github/SAZZZO99/MNIST-DATASET/blob/master/MNIST_LDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage

In [33]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
x = mnist.data
y = mnist.target

In [34]:
e_k = np.zeros_like(x)
s_k = np.zeros_like(x)
n_k = np.zeros_like(x)
nw_k = np.zeros_like(x)
ne_k = np.zeros_like(x)
sw_k = np.zeros_like(x)
se_k = np.zeros_like(x)
w_k = np.zeros_like(x)

In [35]:
ka= np.array([[-3,-3,-3],[-3,0,-3],[5,5,5]])
na= np.array([[-3,-3,5],[-3,0,5],[-3,-3,5]])
wa= np.array([[5,5,5],[-3,0,-3],[-3,-3,-3]])
sa= np.array([[5,-3,-3],[5,0,-3],[5,-3,-3]])
nea= np.array([[-3,-3,-3],[-3,0,5],[-3,5,5]])
nwa= np.array([[-3,5,5],[-3,0,5],[-3,-3,-3]])
sea= np.array([[-3,-3,-3],[5,0,-3],[5,5,-3]])
swa= np.array([[5,5,-3],[5,0,-3],[-3,-3,-3]])

In [36]:
for i in range(70000):
  e_k[i]=ndimage.convolve(x[i].reshape((28, 28)),ka,mode='nearest',cval=0.0).reshape(784)
  s_k[i]=ndimage.convolve(x[i].reshape((28, 28)),sa,mode='nearest',cval=0.0).reshape(784)
  n_k[i]=ndimage.convolve(x[i].reshape((28, 28)),na,mode='nearest',cval=0.0).reshape(784)
  w_k[i]=ndimage.convolve(x[i].reshape((28, 28)),wa,mode='nearest',cval=0.0).reshape(784)
  nw_k[i]=ndimage.convolve(x[i].reshape((28, 28)),nwa,mode='nearest',cval=0.0).reshape(784)
  ne_k[i]=ndimage.convolve(x[i].reshape((28, 28)),nea,mode='nearest',cval=0.0).reshape(784)
  sw_k[i]=ndimage.convolve(x[i].reshape((28, 28)),swa,mode='nearest',cval=0.0).reshape(784)
  se_k[i]=ndimage.convolve(x[i].reshape((28, 28)),sea,mode='nearest',cval=0.0).reshape(784)
 


In [37]:
ldp_mat=np.zeros_like(x)

In [38]:
ldp_hist=np.zeros((70000,56))

In [39]:
for i in range(70000):
  e=e_k[i].reshape((28,28))
  s=s_k[i].reshape((28,28))
  n=n_k[i].reshape((28,28))
  w=w_k[i].reshape((28,28))
  nw=nw_k[i].reshape((28,28))
  ne=ne_k[i].reshape((28,28))
  sw=sw_k[i].reshape((28,28))
  se=se_k[i].reshape((28,28))
  ldp=ldp_mat[i].reshape((28,28))

  for k in range(28):
    for j in range(28):
      lst=[se[k][j],s[k][j],sw[k][j],w[k][j],nw[k][j],n[k][j],ne[k][j],e[k][j]]
      l=[abs(h) for h in lst]
      marr=np.argsort(l)
      marr1=marr[::-1]
      binary=np.zeros(8,dtype="uint8")
      binary[marr1[0]]=1
      binary[marr1[1]]=1
      binary[marr1[2]]=1
      d_no=binary[0]*2**7+binary[1]*2**6+binary[2]*2**5+binary[3]*2**4+binary[4]*2**3+binary[5]*2**2+binary[6]*2**1+binary[7]*2**0
      ldp[k][j]=d_no
  
  ldp_mat[i]=ldp.reshape(784)



In [40]:
for i in range (70000):
  hist=ldp_mat[i].reshape((28,28))
  arr=np.zeros(56)
  for c in range(1,57):
    cnt=0
    for k in range(28):
      for j in range(28):
        if hist[k][j]==c:
          cnt+=1
    arr[c-1]=cnt
  ldp_hist[i]=arr

In [41]:

from sklearn.model_selection import train_test_split
train_img, test_img, train_lbl, test_lbl = train_test_split( ldp_hist, mnist.target, test_size=1/7.0, random_state=0)

In [42]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [43]:
scaler.fit(train_img)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [44]:
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

In [45]:
from sklearn.svm import SVC

In [46]:
svc_model=SVC()

In [47]:
import time

In [48]:
f=time.time()
svc_model.fit(train_img, train_lbl)
q=time.time()
print(q-f)

361.4828128814697


In [49]:
y_predict=svc_model.predict(test_img)

In [50]:
from sklearn import metrics

In [51]:
d=svc_model.score(test_img,test_lbl)

In [52]:
print(d*100)

64.02


In [53]:
t=svc_model.score(train_img,train_lbl)

In [54]:
t=svc_model.score(train_img,train_lbl)

In [55]:
print(t*100)

68.71833333333333
